### Задача 1.

Используя данные из таблицы `user_actions`, рассчитайте среднее число заказов всех пользователей нашего сервиса.

Для этого сначала в подзапросе посчитайте, сколько заказов сделал каждый пользователь, а затем обратитесь к результату подзапроса в блоке `FROM` и уже в основном запросе усредните количество заказов по всем пользователям.

Полученное среднее число заказов всех пользователей округлите до двух знаков после запятой. Колонку с этим значением назовите `orders_avg`.

Поле в результирующей таблице: `orders_avg`

```SQL
SELECT ROUND(AVG(orders_count), 2) AS orders_avg
FROM (
    SELECT user_id, COUNT(order_id) AS orders_count
    FROM user_actions
    WHERE action != 'cancel_order'
    GROUP BY user_id
) AS subquery
```

### Задача 2.

Повторите запрос из предыдущего задания, но теперь вместо подзапроса используйте оператор `WITH` и табличное выражение.

Условия задачи те же: используя данные из таблицы `user_actions`, рассчитайте среднее число заказов всех пользователей.

Полученное среднее число заказов округлите до двух знаков после запятой. Колонку с этим значением назовите `orders_avg`.

Поле в результирующей таблице: `orders_avg`

```SQL
WITH 
subquery AS (
    SELECT user_id, COUNT(order_id) AS orders_count
    FROM user_actions
    WHERE action != 'cancel_order'
    GROUP BY user_id
)

SELECT ROUND(AVG(orders_count), 2) AS orders_avg
FROM subquery
```

### Задача 3.

Выведите из таблицы `products` информацию о всех товарах кроме самого дешёвого.

Результат отсортируйте по убыванию `id` товара.

Поля в результирующей таблице: `product_id`, `name`, `price`

```SQL
SELECT product_id, name, price
FROM products
WHERE price != (SELECT MIN(price) FROM products)
ORDER BY product_id DESC
```

### Задача 4.

Выведите информацию о товарах в таблице `products`, цена на которые превышает среднюю цену всех товаров на 20 рублей и более. Результат отсортируйте по убыванию `id` товара.

Поля в результирующей таблице: `product_id`, `name`, `price`

```SQL
SELECT product_id, name, price
FROM products
WHERE price > (SELECT AVG(price) FROM products) + 20
ORDER BY product_id DESC
```

### Задача 5.

Посчитайте количество уникальных клиентов в таблице `user_actions`, сделавших за последнюю неделю хотя бы один заказ.

Полученную колонку с числом клиентов назовите `users_count`. В качестве текущей даты, от которой откладывать неделю, используйте последнюю дату в той же таблице `user_actions`.

Поле в результирующей таблице: `users_count`

```SQL
SELECT CONUT(DISTINCT user_id) AS users_count
FROM   user_actions
WHERE  action = 'create_order'
   and time >= (SELECT MAX(time) FROM user_actions) - INTERVAL '7 day'
```

### Задача 6.

С помощью функции `AGE` и агрегирующей функции снова определите возраст самого молодого курьера мужского пола в таблице couriers, но в этот раз при расчётах в качестве первой даты используйте последнюю дату из таблицы `courier_actions`.

Чтобы получить именно дату, перед применением функции `AGE` переведите последнюю дату из таблицы `courier_actions` в формат `DATE`, как мы делали в этом задании.

Возраст курьера измерьте количеством лет, месяцев и дней и переведите его в тип `VARCHAR`. Полученную колонку со значением возраста назовите `min_age`.

Поле в результирующей таблице: `min_age`

```SQL
SELECT
  AGE(
    (
      SELECT
        MAX(time)
      FROM
        courier_actions
    )::DATE,
    birth_date
  )::VARCHAR AS min_age
FROM
  couriers
WHERE
  sex = 'male'
ORDER BY
  min_age
limit
  1
```

### Задача 7.

Из таблицы `user_actions` с помощью подзапроса или табличного выражения отберите все заказы, которые не были отменены пользователями.

Выведите колонку с `id` этих заказов. Результат запроса отсортируйте по возрастанию `id` заказа.

Добавьте в запрос оператор `LIMIT` и выведите только первые 1000 строк результирующей таблицы.

Поле в результирующей таблице: `order_id`

```SQL
SELECT
    DISTINCT order_id
FROM
    user_actions
WHERE
    order_id NOT IN (SELECT order_id FROM user_actions WHERE action = 'cancel_order')
ORDER BY
    order_id
LIMIT 1000
```

### Задача 8.

Используя данные из таблицы `user_actions`, рассчитайте, сколько заказов сделал каждый пользователь и отразите это в столбце `orders_count`.

В отдельном столбце `orders_avg` напротив каждого пользователя укажите среднее число заказов всех пользователей, округлив его до двух знаков после запятой.

Также для каждого пользователя посчитайте отклонение числа заказов от среднего значения. Отклонение считайте так: число заказов «минус» округлённое среднее значение. Колонку с отклонением назовите `orders_diff`.

Результат отсортируйте по возрастанию `id` пользователя. Добавьте в запрос оператор `LIMIT` и выведите только первые 1000 строк результирующей таблицы.

Поля в результирующей таблице: `user_id`, `orders_count`, `orders_avg`, `orders_diff`

```SQL
SELECT
    user_id,
    COUNT(DISTINCT order_id) AS orders_count,
    ROUND((SELECT COUNT(DISTINCT order_id) /
    COUNT(DISTINCT user_id)::DECIMAL FROM user_actions), 2) AS orders_avg,
    COUNT(DISTINCT order_id) - ROUND((SELECT COUNT(DISTINCT order_id) /
    COUNT(DISTINCT user_id)::DECIMAL FROM user_actions), 2) AS orders_diff
FROM
    user_actions
GROUP BY 
    user_id
ORDER BY 
    user_id
LIMIT 1000
```

### Задача 9.

Назначьте скидку 15% на товары, цена которых превышает среднюю цену на все товары на 50 и более рублей, а также скидку 10% на товары, цена которых ниже средней на 50 и более рублей. Цену остальных товаров внутри диапазона (среднее - 50; среднее + 50) оставьте без изменений. При расчёте средней цены, округлите её до двух знаков после запятой.

Выведите информацию о всех товарах с указанием старой и новой цены. Колонку с новой ценой назовите `new_price`.

Результат отсортируйте сначала по убыванию прежней цены в колонке `price`, затем по возрастанию `id` товара.

Поля в результирующей таблице: `product_id`, `name`, `price`, `new_price`

```SQL
WITH
avg_price AS (SELECT ROUND(AVG(price), 2) FROM products)

SELECT 
    product_id,
    name,
    price,
    CASE 
        WHEN price >= (SELECT * FROM avg_price) + 50 THEN price * .85
        WHEN price <= (SELECT * FROM avg_price) - 50 THEN price * .9
        ELSE price
    END AS new_price
FROM products
ORDER BY 
    price DESC,
    product_id
```

### Задача 10.

Выясните, есть ли в таблице `courier_actions` такие заказы, которые были приняты курьерами, но не были созданы пользователями. Посчитайте количество таких заказов.

Колонку с числом заказов назовите `orders_count`.

Поле в результирующей таблице: `orders_count`

```SQL
SELECT 
    COUNT(DISTINCT order_id) AS orders_count
FROM courier_actions
WHERE action = 'accept_order' AND order_id NOT IN (SELECT order_id FROM user_actions)
```

### Задача 11.

Выясните, есть ли в таблице `courier_actions` такие заказы, которые были приняты курьерами, но не были доставлены пользователям. Посчитайте количество таких заказов.

Колонку с числом заказов назовите `orders_count`.

Поле в результирующей таблице: `orders_count`

```SQL
SELECT 
    COUNT(DISTINCT order_id) AS orders_count
FROM courier_actions
WHERE action = 'accept_order' 
    AND order_id NOT IN 
    (SELECT order_id FROM courier_actions WHERE action = 'deliver_order')
```

### Задача 12.

Определите количество отменённых заказов в таблице `courier_actions` и выясните, есть ли в этой таблице такие заказы, которые были отменены пользователями, но при этом всё равно были доставлены. Посчитайте количество таких заказов.

Колонку с отменёнными заказами назовите `orders_canceled`. Колонку с отменёнными, но доставленными заказами назовите `orders_canceled_and_delivered.` 

Поля в результирующей таблице: `orders_canceled`, `orders_canceled_and_delivered`

```SQL
SELECT
    COUNT(DISTINCT order_id) 
        FILTER(WHERE order_id IN (SELECT order_id FROM user_actions WHERE action = 'cancel_order')) AS orders_canceled,
    COUNT(DISTINCT order_id) 
        FILTER(WHERE action = 'deliver_order' 
        AND order_id IN (SELECT order_id FROM user_actions WHERE action = 'cancel_order')) AS orders_canceled_and_delivered
FROM courier_actions
```

### Задача 13.

По таблицам `courier_actions` и `user_actions` снова определите число недоставленных заказов и среди них посчитайте количество отменённых заказов и количество заказов, которые не были отменены (и соответственно, пока ещё не были доставлены).

Колонку с недоставленными заказами назовите `orders_undelivered`, колонку с отменёнными заказами назовите `orders_canceled`, колонку с заказами «в пути» назовите `orders_in_process`.

Поля в результирующей таблице: `orders_undelivered`, `orders_canceled`, `orders_in_process`

```SQL
SELECT
    COUNT(DISTINCT order_id) AS orders_undelivered,
    COUNT(DISTINCT order_id) FILTER(WHERE order_id IN (SELECT order_id FROM user_actions WHERE action = 'cancel_order')) AS orders_canceled,
    COUNT(DISTINCT order_id) FILTER(WHERE order_id NOT IN (SELECT order_id FROM user_actions WHERE action = 'cancel_order')) AS orders_in_process
FROM courier_actions
WHERE order_id NOT IN (SELECT order_id FROM courier_actions WHERE action = 'deliver_order')
```

### Задача 14.

Отберите из таблицы users пользователей мужского пола, которые старше всех пользователей женского пола.

Выведите две колонки: `id` пользователя и дату рождения. Результат отсортируйте по возрастанию `id` пользователя.

Поля в результирующей таблице: `user_id`, `birth_date`

```SQL
SELECT
    user_id,
    birth_date
FROM 
    users
WHERE 
    sex = 'male'
    AND birth_date < (SELECT MIN(birth_date) FROM users WHERE sex = 'female')
ORDER BY 
    user_id
```

### Задача 15.

Выведите `id` и содержимое 100 последних доставленных заказов из таблицы `orders`.

Содержимым заказов считаются списки с `id` входящих в заказ товаров. Результат отсортируйте по возрастанию `id` заказа.

Поля в результирующей таблице: `order_id`, `product_ids`

```SQL
SELECT
    order_id,
    product_ids
FROM 
    orders
WHERE 
    order_id IN (SELECT order_id
                 FROM courier_actions
                 WHERE action = 'deliver_order'
                 ORDER BY time DESC
                 LIMIT 100)
ORDER BY
    order_id
```

### Задача 16.

Из таблицы `couriers` выведите всю информацию о курьерах, которые в сентябре 2022 года доставили 30 и более заказов. Результат отсортируйте по возрастанию `id` курьера.

Поля в результирующей таблице: `courier_id`, `birth_date`, `sex`

```SQL
SELECT
    courier_id,
    birth_date,
    sex
FROM 
    couriers
WHERE 
    courier_id IN (
        SELECT courier_id
        FROM courier_actions
        WHERE action = 'deliver_order' 
        AND DATE_PART('month', time) = 9
        AND DATE_PART('year', time) = 2022
        GROUP BY courier_id
        HAVING COUNT(action) >= 30
        )
ORDER BY
    courier_id
```

### Задача 17.

Рассчитайте средний размер заказов, отменённых пользователями мужского пола.

Средний размер заказа округлите до трёх знаков после запятой. Колонку со значением назовите `avg_order_size`.

Поле в результирующей таблице: `avg_order_size`

```SQL
SELECT
    ROUND(AVG(array_length(product_ids, 1)), 3) AS avg_order_size
FROM
    orders
WHERE
    order_id IN (
    SELECT order_id
    FROM user_actions
    WHERE action = 'cancel_order'
    AND user_id IN (
        SELECT user_id
        FROM users
        WHERE sex = 'male'
    )
)
```

### * Задача 18.

Посчитайте возраст каждого пользователя в таблице `users`.

Возраст измерьте числом полных лет, как мы делали в прошлых уроках. Возраст считайте относительно последней даты в таблице `user_actions`.

Для тех пользователей, у которых в таблице users не указана дата рождения, укажите среднее значение возраста всех остальных пользователей, округлённое до целого числа.

Колонку с возрастом назовите `age`. В результат включите колонки с `id` пользователя и возрастом. Отсортируйте полученный результат по возрастанию `id` пользователя.

Поля в результирующей таблице: `user_id`, `age`

```SQL
SELECT user_id,
    COALESCE(
    DATE_PART('year', AGE((SELECT MAX(time) FROM user_actions), birth_date))::INTEGER, 
    (SELECT ROUND(AVG(DATE_PART('year', AGE((SELECT MAX(time) FROM user_actions), birth_date)))) FROM users)::INTEGER
    ) AS age
FROM users
ORDER BY user_id
```

### * Задача 19.

Для каждого заказа, в котором больше 5 товаров, рассчитайте время, затраченное на его доставку. 

В результат включите `id` заказа, время принятия заказа курьером, время доставки заказа и время, затраченное на доставку. Новые колонки назовите соответственно `time_accepted`, `time_delivered` и `delivery_time`.

В расчётах учитывайте только неотменённые заказы. Время, затраченное на доставку, выразите в минутах, округлив значения до целого числа. Результат отсортируйте по возрастанию `id` заказа.

Поля в результирующей таблице: `order_id`, `time_accepted`, `time_delivered` и `delivery_time`

```SQL
SELECT
    order_id,
    time_accepted,
    time_delivered,
    ROUND(EXTRACT(epoch FROM time_delivered - time_accepted)/60)::INTEGER AS delivery_time
FROM
    (
    SELECT 
        order_id,
        MAX(CASE WHEN action = 'accept_order' THEN time ELSE NULL END) AS time_accepted,
        MAX(CASE WHEN action = 'deliver_order' THEN time ELSE NULL END) AS time_delivered
    FROM courier_actions 
    WHERE order_id NOT IN (SELECT order_id FROM user_actions WHERE action = 'cancel_order')
    AND order_id IN (SELECT
            order_id
        FROM orders
        WHERE array_length(product_ids, 1) > 5)
    GROUP BY order_id
    ) AS t
ORDER BY 
    order_id
```

### * Задача 20.

Для каждой даты в таблице `user_actions` посчитайте количество первых заказов, совершённых пользователями.

Первыми заказами будем считать заказы, которые пользователи сделали в нашем сервисе впервые. В расчётах учитывайте только неотменённые заказы.

В результат включите две колонки: дату и количество первых заказов в эту дату. Колонку с датами назовите `date`, а колонку с первыми заказами — `first_orders`.

Результат отсортируйте по возрастанию даты.

Поля в результирующей таблице: `date`, `first_orders`

```SQL
WITH not_canceled AS (
    SELECT order_id
    FROM   user_actions
    WHERE  order_id NOT IN (SELECT order_id FROM   user_actions WHERE  action = 'cancel_order')
    )
            
SELECT DATE(time) AS date,
    COUNT(*) AS first_orders
FROM user_actions
WHERE order_id IN (SELECT MIN(order_id)
                FROM user_actions
                WHERE order_id IN (SELECT * FROM not_canceled)
                GROUP BY user_id)
GROUP BY date
ORDER BY date
```

### Задача 21.

Выберите все колонки из таблицы `orders` и дополнительно в качестве последней колонки укажите функцию `unnest`, применённую к колонке `product_ids`. Эту последнюю колонку назовите `product_id`. Больше ничего с данными делать не нужно.

Добавьте в запрос оператор `LIMIT` и выведите только первые 100 записей результирующей таблицы.

Поля в результирующей таблице: `creation_time`, `order_id`, `product_ids`, `product_id`

Посмотрите на результат работы функции `unnest` и постарайтесь разобраться, что произошло с исходной таблицей.

```SQL
SELECT 
    creation_time,
    order_id,
    product_ids,
    unnest(product_ids) AS product_id
FROM orders
LIMIT 100
```

### * Задача 22.

Используя функцию `unnest`, определите 10 самых популярных товаров в таблице `orders`.

Самыми популярными товарами будем считать те, которые встречались в заказах чаще всего. Если товар встречается в одном заказе несколько раз (когда было куплено несколько единиц товара), это тоже учитывается при подсчёте. Учитывайте только неотменённые заказы.

Выведите `id` товаров и то, сколько раз они встречались в заказах (то есть сколько раз были куплены). Новую колонку с количеством покупок товаров назовите `times_purchased`.

Результат отсортируйте по возрастанию `id` товара.

Поля в результирующей таблице: `product_id`, `times_purchased`

```SQL
SELECT
    * 
FROM
    (SELECT
        unnest(product_ids) AS product_id,
        COUNT(*) AS times_purchased
    FROM orders
    WHERE order_id NOT IN (SELECT order_id FROM   user_actions WHERE  action = 'cancel_order')
    GROUP BY unnest(product_ids)
    ORDER BY times_purchased DESC
    LIMIT 10
    ) AS t
ORDER BY product_id
```

### * Задача 23.

Из таблицы `orders` выведите `id` и содержимое заказов, которые включают хотя бы один из пяти самых дорогих товаров, доступных в нашем сервисе.

Результат отсортируйте по возрастанию `id` заказа.

Поля в результирующей таблице: `order_id`, `product_ids`

```SQL
SELECT
    order_id,
    product_ids
FROM 
    (SELECT 
        order_id,
        product_ids,
        unnest(product_ids) AS product_id
    FROM 
        orders) t
WHERE product_id IN (SELECT product_id FROM products ORDER BY price DESC LIMIT 5)
GROUP BY 
    1, 2
ORDER BY 
    order_id
```

### Подведём итоги

В этом уроке мы:

- Систематизировали накопленные знания и окончательно разобрались с порядком выполнения операторов в запросах.
- Научились составлять подзапросы и узнали, что их можно применять в блоках <span style="color:Tomato">**SELECT**</span>, <span style="color:Tomato">**FROM**</span>, <span style="color:Tomato">**WHERE**</span> и <span style="color:Tomato">**HAVING**</span>.
- Познакомились с табличными выражениями и оператором <span style="color:Tomato">**WITH**</span>, научились писать более аккуратный и понятный код.
- Ещё немного поработали с датами, изучили функцию <span style="color:Tomato">**NOW**</span> и узнали, как проводить арифметические операции с интервалами.
- Познакомились с <span style="color:Tomato">**EXTRACT**</span> — ещё одной функцией для работы с датами.
- Поработали с новой функцией <span style="color:Tomato">**unnest**</span> и научились разворачивать списки со значениями в расширенные таблицы.
- Решили несколько сложных задач на подзапросы и табличные выражения с применением всех полученных знаний.


Известные нам на текущий момент ключевые слова и порядок исполнения в запросе:

>```SQL
>FROM       -- указание источника данных
>WHERE      -- фильтрация данных
>GROUP BY   -- группировка данных
>HAVING     -- фильтрация данных после группировки
>SELECT     -- перечисление полей результирующей таблицы
>ORDER BY   -- сортировка результирующей таблицы
>LIMIT      -- ограничение количества выводимых записей
>```